## Data Crawling
- 사이트별로 직무내용이 너무 상이함, 구분을 지어주기가 쉽지 않음  
-> 학습에 의해 판단되므로 직무내용을 굳이 안넣어도 될 것 같음  
-> ```회사소개, 주요업무 column 추가``` 아래의 내용과 같이 모든 column 통일  
```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [1]:
import pandas as pd
import numpy as np
import re

### 원티드 데이터 전처리
- column확인 후 필요없는 column제거 후 정해진 frame의 column name으로 변경
- 정해진 frame의 column_name으로 변경
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [3]:
wanted = pd.read_csv('../Crawling/sample_data/wanted.csv')
wanted.head(1)

,직무명,채용공고명,해시태그,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,근무지역,회사명,회사분야,출처
0,웹 개발자,Python Developer (백엔드 주니어),#퇴사율5%이하#51~300명#설립4~9년#육아휴직#출산휴가#스톡옵션#건강검진#교육...,"- 간단소개\r\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 Fa...",• 세무 알고리즘 자동화 설계 및 개발\r\n• 세무팀과 커뮤니케이션\r\n• AW...,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...","• MSA 기반의 시스템 개발 경험이 있으신 분\r\n• AWS 를 활용한 개발, ...",# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...,Github\r\nJavaScript\r\nTypeScript\r\nReact.js...,서울.한국,혜움랩스,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167436


- **결측치 확인 부분**  
상황에 따라 다르게 작성

In [4]:
#우대사항, 자격요건, 혜택 및 복지 부분 nan값 row 제거

wanted.dropna(subset=['우대사항', '자격요건','혜택 및 복지'], inplace=True)
wanted.count()

직무명         7874
채용공고명       7858
해시태그        7874
회사소개        7874
주요업무        7874
자격요건        7874
우대사항        7874
혜택 및 복지     7874
기술스택 ・ 툴    5695
근무지역        7874
회사명         7874
회사분야        7874
출처          7874
dtype: int64

In [7]:
#채용공고명만 비어있는 부분 채워줌, 크롤링 진행시 누락된 것으로 확인

null_data = wanted[wanted['채용공고명'].isnull() == True]['출처'].tolist()
add_data = ['[제주] 백엔드 개발자 (5년 이하)',
            '백엔드 개발자',
            '[광고 제품팀] 풀 스택 엔지니어 리드',
            'Sr. Back-end Engineer',
            'iOS 코어 개발자 (5년 이상)',
            '모바일 SW 개발자(C,C++)',
            '디바이스 부문 개발자',
            'Unity 개발자',
            '기술연구소 백엔드 서버 개발자',
            '백엔드 개발자 (BackEnd Engineer)',
            'Senior Software Engineer, Android',
            '백엔드 개발자(Node.js, 시니어)',
            '백엔드 개발자 (Node.js, 주니어)',
            0,
            'AI SW 개발자',
            'Data Engineer(2년 이상)']

for n_data, a_data in zip(null_data,add_data):
    if a_data == 0:
        wanted.drop(wanted[wanted['출처'] == n_data].index, inplace=True)
        continue
    wanted.loc[wanted['출처'] == n_data, '채용공고명'] = a_data

#drop으로 인한 index 누락 reindexing
wanted.reset_index(drop=True, inplace=True)
wanted.count()

직무명         7873
채용공고명       7873
해시태그        7873
회사소개        7873
주요업무        7873
자격요건        7873
우대사항        7873
혜택 및 복지     7873
기술스택 ・ 툴    5695
근무지역        7873
회사명         7873
회사분야        7873
출처          7873
dtype: int64

In [8]:
wanted.rename(columns={'직무명':'직무내용',
                       '채용공고명':'공고명',
                       '기술스택 ・ 툴':'기술 스택',
                       '혜택 및 복지':'복지',
                       '출처':'출처 URL',
                       '우대사항':'우대조건',
                       }, inplace=True)

wanted.columns

Index(['직무내용', '공고명', '해시태그', '회사소개', '주요업무', '자격요건', '우대조건', '복지', '기술 스택',
       '근무지역', '회사명', '회사분야', '출처 URL'],
      dtype='object')

- 지원기간은 이야기 해봐야 할 듯, 경력조건은 자격요건의 내용에서 추출

### wanted 패턴
- 경력 n년 이상
- 경력 n년이상
- n~k년
- n년~k년

In [43]:
def extract_years(string, patterns):
    for pattern in patterns:
        match = re.search(pattern, string)
        if match:
            if '~' in pattern:
                start_year = match.group(1)
                return int(start_year)
            else:
                return int(match.group(1))
    
    return 0


In [10]:
max_count = wanted['자격요건'].count()
wanted['경력조건'] = None
patterns = [r'(\d+)년 이상', r'(\d+)년이상', r'(\d+)~(\d+)년', r'(\d+)년~(\d+)년']

for i in range(0, max_count):
    wanted.loc[i, '경력조건'] = extract_years(wanted['자격요건'][i], patterns)


In [11]:
wanted['경력조건'].unique()

array([2, 0, 3, 1, 7, 6, 4, 5, 8, 10, 15, 9, 12], dtype=object)

- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [14]:
new_order = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지', '회사소개', '주요업무']
wanted = wanted.reindex(columns=new_order)
wanted.count()

회사명       7873
지원기간         0
공고명       7873
직무내용      7873
자격요건      7873
우대조건      7873
경력조건      7873
기술 스택     5695
출처 URL    7873
복지        7873
회사소개      7873
주요업무      7873
dtype: int64

In [15]:
#csv 파일로 저장

wanted.to_csv('./Data_result/pre_wanted.csv',index=False)

### 잡플래닛 데이터 전처리
- column확인 후 필요없는 column제거 후 정해진 frame의 column name으로 변경
- 정해진 frame의 column_name으로 변경
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [106]:
jobplanet = pd.read_csv('../Crawling/Announcement_data/extracted_softwareengineer.csv')
jobplanet.head(1)

,href,title,company_name,job_location,job_classification,due_date,work_experience,skills,company_intro,main_task,qualification,preference,procedure,benefits,job_location_detail,detail_description
0,https://www.jobplanet.co.kr/job/search?posting...,B2B 백엔드 엔지니어,(주)마이프랜차이즈,서울,백엔드 개발,2023.07.24 D-1,경력무관,"백엔드, B2B, MongoDB, node js",＜마이프랜차이즈가 오프라인 창업 시장을 바꿉니다.＞\r\n\r\n코로나19 이후 어...,• 마이프차 Partner 대상 서비스 백엔드 개선 및 새 기능 개발\r\n• 각...,• 3년 이상의 상용 서비스 백엔드 개발 경험 또는 이에 준하는 역량 보유자\r\n...,• 프로젝트 세팅부터 서비스 런칭까지 전 과정에 대한 경험이 있는 분\r\n• MS...,＜채용은 이렇게 진행됩니다＞\r\n1. 서류 전형\r\n : 마이프랜차이즈는 정해진...,"• 시차출퇴근제 (08~10시 자율 출근, 일 8시간 근무)\r\n• 수습 기간 종...","서울시 강남구 강남대로92길 19, 여원빌딩 11층","<section class=""recruitment-detail""><div class..."


In [107]:
jobplanet.rename(columns={'href': '출처 URL', 
                          'title': '공고명',
                          'company_name':'회사명',
                          'job_classification':'직무내용',
                          'due_date':'지원기간',
                          'work_experience':'경력조건',
                          'skills':'기술 스택',
                          'main_task':'주요 업무',
                          'qualification':'자격요건',
                          'preference':'우대조건',
                          'benefits':'복지',
                          'company_intro': '회사소개',
                          'main_task':'주요업무'
                          }, inplace=True)
jobplanet.columns

Index(['출처 URL', '공고명', '회사명', 'job_location', '직무내용', '지원기간', '경력조건', '기술 스택',
       '회사소개', '주요업무', '자격요건', '우대조건', 'procedure', '복지',
       'job_location_detail', 'detail_description'],
      dtype='object')

In [108]:
new_order = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지','회사소개','주요업무']
jobplanet = jobplanet.reindex(columns=new_order)
jobplanet.head(1)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지,회사소개,주요업무
0,(주)마이프랜차이즈,2023.07.24 D-1,B2B 백엔드 엔지니어,백엔드 개발,• 3년 이상의 상용 서비스 백엔드 개발 경험 또는 이에 준하는 역량 보유자\r\n...,• 프로젝트 세팅부터 서비스 런칭까지 전 과정에 대한 경험이 있는 분\r\n• MS...,경력무관,"백엔드, B2B, MongoDB, node js",https://www.jobplanet.co.kr/job/search?posting...,"• 시차출퇴근제 (08~10시 자율 출근, 일 8시간 근무)\r\n• 수습 기간 종...",＜마이프랜차이즈가 오프라인 창업 시장을 바꿉니다.＞\r\n\r\n코로나19 이후 어...,• 마이프차 Partner 대상 서비스 백엔드 개선 및 새 기능 개발\r\n• 각...


- **결측치 확인 부분**  
```count()로 결측치 부분 확인 후 삭제 or 채움 결정```  
-> 데이터가 많지 않다면 삭제하는 방향

In [109]:
jobplanet.count()

회사명       1242
지원기간      1242
공고명       1242
직무내용      1242
자격요건      1242
우대조건      1215
경력조건      1242
기술 스택     1242
출처 URL    1242
복지        1230
회사소개      1233
주요업무      1242
dtype: int64

In [97]:
# 결측치의 내용에 따라 변경 
jobplanet.drop(jobplanet[jobplanet['복지'].isnull() == True].index, inplace=True)
jobplanet.drop(jobplanet[jobplanet['우대조건'].isnull() == True].index, inplace=True)
jobplanet.reset_index(drop=True, inplace=True)
jobplanet.count()

회사명       127
지원기간      127
공고명       127
직무내용      127
자격요건      127
우대조건      127
경력조건      127
기술 스택     127
출처 URL    127
복지        127
회사소개      127
주요업무      127
dtype: int64

In [52]:
for i, data in jobplanet[jobplanet['회사소개'].isnull()==True]['출처 URL'].items():
    print(i, data)

100 https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1251837
101 https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1251832
117 https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1251828
381 https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250583
554 https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1247896
814 https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1245673


In [54]:
jobplanet.drop(jobplanet[jobplanet['회사소개'].isnull() == True].index, inplace=True)
jobplanet.reset_index(drop=True, inplace=True)
jobplanet.count()

회사명       1199
지원기간      1199
공고명       1199
직무내용      1199
자격요건      1199
우대조건      1199
경력조건      1199
기술 스택     1199
출처 URL    1199
복지        1199
회사소개      1199
주요업무      1199
dtype: int64

- 경력조건 int형 변환
- wanted data 처리 때 만들었던 extract_years()를 사용한다.

In [98]:
for i, data in jobplanet[jobplanet['경력조건'] == '신입, 3년 이상']['출처 URL'].items():
    print(i, data)

In [99]:
jobplanet['경력조건'].unique()

array(['경력무관', '1년 이상', '7년 이상', '3년 이상', '5년 이상', '6년 이상', '2년 이상',
       '4년 이상', '신입', '10년 이상', '신입, 1년 이상', '신입, 경력무관', '5년 이상, 경력무관',
       '신입, 3년 이상, 경력무관'], dtype=object)

In [100]:
#정규화 패턴으로 경력조건 -> int형으로 변환하는 코드

def extract_years(string, patterns):
    for pattern in patterns:
        match = re.search(pattern, string)
        if match:
            if '~' in pattern:
                start_year = match.group(1)
                return int(start_year)
            else:
                return int(match.group(1))
    
    return 0

In [101]:
max_count = jobplanet['경력조건'].count()
patterns = [r'(\d+)년 이상']

for i in range(0, max_count):
    jobplanet.loc[i, '경력조건'] = extract_years(jobplanet['경력조건'][i], patterns)

jobplanet['경력조건'].unique()

array([0, 1, 7, 3, 5, 6, 2, 4, 10], dtype=object)

- 지원기간 ```YYYY.MM.DD``` 형태로 변환

In [102]:
jobplanet['지원기간'] = jobplanet['지원기간'].str.split('\xa0').str[0]
jobplanet['지원기간'].unique()

array(['2023.08.21', '2023.10.12', '2023.10.21', '2023.08.20',
       '2023.07.31', '2023.08.14', '2023.10.19', '2023.07.30',
       '2023.10.14', '2023.08.11', '2023.10.11', '2023.10.07',
       '2023.08.16', '2023.09.04', '2023.08.07', '2023.08.03',
       '2023.10.06', '2023.10.04', '2023.09.30', '2023.08.31',
       '2023.07.28', '2023.09.27', '2023.09.26', '2023.10.25',
       '2023.10.01', '2023.09.20', '2023.09.28', '2023.09.15',
       '2023.09.14', '2023.09.09', '2023.08.09', '2023.09.07',
       '2023.09.13', '2023.08.27', '2023.08.22', '2023.08.29',
       '2023.08.24', '2023.08.19', '2023.10.10', '2023.11.28',
       '2023.09.24', '2023.09.06', '2023.08.10', '2023.08.17',
       '2023.10.15', '2023.09.21', '2023.09.02', '2023.08.18',
       '2023.09.11', '2023.10.09', '2023.08.26', '2023.08.12',
       '2023.08.25', '2023.10.29', '2023.08.23', '2023.10.17'],
      dtype=object)

In [103]:
jobplanet.count()

회사명       127
지원기간      127
공고명       127
직무내용      127
자격요건      127
우대조건      127
경력조건      127
기술 스택     127
출처 URL    127
복지        127
회사소개      127
주요업무      127
dtype: int64

In [104]:
before_result = pd.read_csv('./Data_result/pre_jobplanet.csv')
print(before_result.count())
result = pd.concat([before_result, jobplanet], axis=0)
result.reset_index(drop=True, inplace=True)
result.count()

회사명       1304
지원기간      1304
공고명       1304
직무내용      1304
자격요건      1304
우대조건      1304
경력조건      1304
기술 스택     1304
출처 URL    1304
복지        1304
회사소개      1304
주요업무      1304
dtype: int64


회사명       1431
지원기간      1431
공고명       1431
직무내용      1431
자격요건      1431
우대조건      1431
경력조건      1431
기술 스택     1431
출처 URL    1431
복지        1431
회사소개      1431
주요업무      1431
dtype: int64

In [105]:
#csv 파일로 저장

result.to_csv('./Data_result/pre_jobplanet.csv',index=False)

### 점핏 데이터 전처리

- 크롤링 문제 확인으로 직무내용 별로 csv 파일 생성 후 concat
- 지원기간과 경력조건 data만 맞추기  
-> 기존 column에서 회사소개, 주요업무 column 추가
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [93]:
jumpit = pd.DataFrame()

for i in range(1, 22):
    if i == 14:
        continue
    df = pd.read_csv(f'../Crawling/jumpit_data/크롤링_점핏_{i}.csv')
    jumpit = pd.concat([jumpit, df])

jumpit.reset_index(drop=True, inplace=True)

In [94]:
jumpit.head(1)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지,회사소개,주요업무
0,"미디어로그는 MVNO(U+유모바일)사업, 중고폰사업(셀로), PP(방송채널)사업과 ...",마감일\n상시,미디어로그(LG그룹) 웹 하드 3.0 SM 경력 채용,서버/백엔드 개발자,"• Java, Spring, Spring Boot, JSP 개발 스킬 필수\n• O...","• LG U+ SI/SM 업무 경험\n• 서버 간 연동 처리 경험 (restful,...",경력\n경력 10~15년,Java\nJSP\nSpring Boot\nMariaDB\nOracle\nLinux,https://www.jumpit.co.kr/position/16607,• 통신비 지원\n- U+유모바일 음성무제한 요금제 지원\n\n• 자녀 학비 지원\...,"미디어로그는 MVNO(U+유모바일)사업, 중고폰사업(셀로), PP(방송채널)사업과 ...",• 웹 하드 3.0 운영 유지보수 기능 개발\n• VOC 대응\n• 장애 대응 및 ...


In [95]:
#크롤링 과정에서 결측치는 없게 처리함

jumpit.count()

회사명       3187
지원기간      3187
공고명       3187
직무내용      3187
자격요건      3187
우대조건      3187
경력조건      3187
기술 스택     3187
출처 URL    3187
복지        3187
회사소개      3187
주요업무      3187
dtype: int64

In [96]:
#마감일\n 삭제와 '-' -> '.' replace

jumpit['지원기간'] = jumpit['지원기간'].str.replace('마감일\n', '').str.replace('-','.')
jumpit['지원기간'].unique()

array(['상시', '2023.07.31', '2023.12.31', '2023.08.31', '2023.08.03',
       '2023.08.07', '2023.08.16', '2023.08.01', '2023.07.21',
       '2023.07.28', '2023.07.23', '2023.11.30', '2023.07.24',
       '2023.08.04', '2023.07.26', '2023.08.02', '2023.07.30',
       '2023.09.07', '2023.07.29', '2023.09.30', '2023.08.11',
       '2023.09.13', '2023.08.21', '2023.09.12', '2023.08.06',
       '2023.10.09', '2023.08.12', '2023.07.27', '2023.10.03',
       '2023.08.23', '2023.08.22', '2023.08.18', '2024.08.14',
       '2023.09.08', '2023.08.17', '2023.08.15', '2023.08.09',
       '2023.10.17', '2023.10.18', '2023.09.11', '2023.08.08',
       '2023.08.28', '2023.08.10', '2023.07.22', '2023.08.13',
       '2023.10.02', '2023.09.29', '2023.10.19', '2023.10.06',
       '2023.09.27', '2023.10.16', '2023.12.01', '2023.08.05',
       '2023.09.06'], dtype=object)

- 경력 조건 부분 ```경력\n``` 제거
- 신입~n년 부분 1~n 년으로 변경
- 경력조건 int형 변환
- wanted data 처리 때 만들었던 extract_years()를 사용한다.

In [100]:
jumpit['경력조건'].unique()

array(['경력\n경력 10~15년', '경력\n경력 3~10년', '경력\n경력 3~6년', '경력\n경력 5~10년',
       '경력\n경력 2~10년', '경력\n경력 1~3년', '경력\n신입~10년', '경력\n경력 2~6년',
       '경력\n경력 3~20년', '경력\n경력 3~12년', '경력\n경력 3~7년', '경력\n경력 1~7년',
       '경력\n경력 3~15년', '경력\n경력 10~20년', '경력\n경력 5~12년', '경력\n경력 5~20년',
       '경력\n신입~3년', '경력\n경력 4~15년', '경력\n신입~2년', '경력\n경력 2~20년',
       '경력\n경력 5~15년', '경력\n경력 4~6년', '경력\n경력 2~7년', '경력\n경력 1~5년',
       '경력\n경력 3~5년', '경력\n경력 2~9년', '경력\n신입~20년', '경력\n경력 7~10년',
       '경력\n경력 7~15년', '경력\n경력 1~10년', '경력\n경력 2~15년', '경력\n신입~15년',
       '경력\n경력 6~20년', '경력\n경력 4~10년', '경력\n경력 1~15년', '경력\n경력 6~15년',
       '경력\n신입~5년', '경력\n경력 2~5년', '경력\n경력 3~8년', '경력\n경력 5~8년',
       '경력\n경력 6~16년', '경력\n경력 6~8년', '경력\n경력 4~11년', '경력\n경력 2~12년',
       '경력\n경력 2~8년', '경력\n경력 8~15년', '경력\n경력 4~7년', '경력\n경력 12~16년',
       '경력\n경력 2~3년', '경력\n경력 5~7년', '경력\n경력 3~9년', '경력\n경력 1~8년',
       '경력\n경력 7~20년', '경력\n경력 10~14년', '경력\n경력 3~4년', '경력\n경력 7~25년',
       '경력\n경력 8~13년', '경력\n경력 4~20년',

In [101]:
max_count = jumpit['경력조건'].count()
jumpit['경력조건'] = jumpit['경력조건'].str.replace('신입~', '0~')
patterns = [r'경력\n경력 (\d+)~(\d+)년',r'경력\n(\d+)~(\d+)년']

for i in range(0, max_count):
    jumpit.loc[i, '경력조건'] = extract_years(jumpit['경력조건'][i], patterns)
    
jumpit['경력조건'].unique()

array([10, 3, 5, 2, 1, 0, 4, 7, 6, 8, 12, 9, 17, 11, 15], dtype=object)

In [102]:
jumpit.to_csv('../Crawling/sample_data/jumpit.csv', index=False)
jumpit.to_csv('./Data_result/pre_jumpit.csv', index=False)

## CSV파일 concat 

- 기술스택, 자격요건 column -> ```자격요건 column으로 concat```
- 현재 코드에서 ```Crawling/sample_data/```에 있는 csv 파일을 통해 ```Data_Pretreatment/Data_result/result.csv```를 만듬

In [12]:
wanted = pd.read_csv('./Data_result/pre_wanted.csv')
jobplanet = pd.read_csv('./Data_result/pre_jobplanet.csv')
jumpit = pd.read_csv('./Data_result/pre_jumpit.csv')

In [13]:
result = pd.concat([wanted, jobplanet, jumpit], axis=0)
result.reset_index(drop=True, inplace=True)
result.count()

회사명       12491
지원기간       4618
공고명       12491
직무내용      12491
자격요건      12491
우대조건      12491
경력조건      12491
기술 스택     10313
출처 URL    12491
복지        12491
회사소개      12491
주요업무      12491
dtype: int64

- 자격요건, 기술 스택 concat
- '자격요건'과 '기술 스택' column의 type을 ```object -> str```변환 후 join을 통해 concat

In [8]:
print(result['자격요건'][0],result['기술 스택'][0])

• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험
• Python(필수), 웹 프레임워크(Django)에 능숙하신 분
• 테스트 코드 작성의 필요성을 이해하고 실천하시는 분
• 코드리뷰와 페어 프로그래밍등 팀 내 협업의 필요성을 이해하고 실천하시는 분
• 개발/기획/세무/운영/CS 등 다양한 직군의 동료들과 원활한 소통이 가능하신 분

Tech Specs
• Language: Python 3
• Framework: Django(rest framework), Flask, FastAPI, Celery
• Data storage: PostgreSQL / Redis / DynamoDB / S3
• Deployment: Github action, Docker(ECS, EKS)
• Cloud platform: AWS
• VCS: Github
• Communication: Slack
• Documentation: Notion, G-Suite, Jira
• Issue Tracker: JIRA Github
JavaScript
TypeScript
React.js
Vue.JS


In [16]:
result['자격요건'] = result['자격요건'].astype(str)
result['기술 스택'] = result['기술 스택'].astype(str)
concat_df = pd.DataFrame({'자격요건': [a+' '+b for a, b in zip(result['자격요건'], result['기술 스택'])]})

result['자격요건'] = concat_df['자격요건']
result.drop(['기술 스택', '지원기간'], axis=1, inplace=True)

In [20]:
result['자격요건'].unique()

array(['• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수), 웹 프레임워크(Django)에 능숙하신 분\r\n• 테스트 코드 작성의 필요성을 이해하고 실천하시는 분\r\n• 코드리뷰와 페어 프로그래밍등 팀 내 협업의 필요성을 이해하고 실천하시는 분\r\n• 개발/기획/세무/운영/CS 등 다양한 직군의 동료들과 원활한 소통이 가능하신 분\r\n\r\nTech Specs\r\n• Language: Python 3\r\n• Framework: Django(rest framework), Flask, FastAPI, Celery\r\n• Data storage: PostgreSQL / Redis / DynamoDB / S3\r\n• Deployment: Github action, Docker(ECS, EKS)\r\n• Cloud platform: AWS\r\n• VCS: Github\r\n• Communication: Slack\r\n• Documentation: Notion, G-Suite, Jira\r\n• Issue Tracker: JIRA Github\r\nJavaScript\r\nTypeScript\r\nReact.js\r\nVue.JS',
       '• HTML / CSS / JavaScript 에 대한 이해\r\n• 컴퓨터 사이언스에 대한 기본지식\r\n• 복잡한 요구사항 속에서 문제를 정의하고 풀어내는 능력\r\n• 익숙하지 않은 도메인(세무, 재무)에 대해서 적극적으로 학습할 수 있는 분\r\n\r\nTech Specs\r\n• JavaScript\r\n• TypeScript\r\n• Vue.js\r\n• Github / Github Action\r\n• Jira\r\n• Slack\r\n• Notion\r\n• AWS Github\r\nJavaScript\r\nTypeScript\r\nReact.js\r\nVue.JS',
       '• 3년 이상 또는 그에 준하는 프론트엔드 서비스 개발 및 유지 관리 경험을

In [19]:
result.count()

회사명       12491
공고명       12491
직무내용      12491
자격요건      12491
우대조건      12491
경력조건      12491
출처 URL    12491
복지        12491
회사소개      12491
주요업무      12491
dtype: int64

---------------------------------------------------

## 추가 데이터 작업시 사용하는 코드
- 3개의 사이트의 기본 전처리는 위의 코드를 참고하여 작성
- 자격요건, 기술스택 concat부분만 작성


In [2]:
data = pd.read_csv('./Data_result/pre_result_morph (1).csv')
data.head(20)

,회사명,공고명,직무내용,자격요건,우대조건,경력조건,출처 URL,복지,회사소개,주요업무
0,혜움랩스,"['Python', 'Developer', '백엔드', '주니어']",웹 개발자,"['최소', '2년', '이상', '혹은', '그', '에', '준', '하는', ...","['MSA', '기반', '의', '시스템', '개발', '경험', '이', '있으...",2,https://www.wanted.co.kr/wd/167436,"['#', '몰입', '을', '위', '한', '액면가', '스톡옵션', '의',...","['간단', '소개', '혜움랩스', '는', '세무', '재무', '경리', '등...","['세무', '알고리즘', '자동화', '설계', '및', '개발', '세무', '..."
1,혜움랩스,"['프론트엔드', '개발자', '주니어']",웹 개발자,"['HTML', 'CSS', 'JavaScript', '에', '대한', '이해',...","['Vue', '.', 'js', 'React', '.', 'js', 'Angula...",0,https://www.wanted.co.kr/wd/167433,"['#', '몰입', '을', '위', '한', '액면가', '스톡옵션', '의',...","['간단', '소개', '혜움랩스', '는', '세무', '재무', '경리', '등...","['세무', '관련', '서비스', '및', '플랫폼', '의', '프론트엔드', ..."
2,유섭,"['프론트엔드', '개발자', 'Software', 'Engineer', '3년',...",웹 개발자,"['3년', '이상', '또는', '그', '에', '준', '하는', '프론트엔드...","['이런', '분', '은', '더욱', '환영', '합니다', '.', '팀', ...",3,https://www.wanted.co.kr/wd/167134,"['자유로운', '휴가', '무제한', '무급', '휴가', '동료', '와', '...","['유섭', '은', '현재', '번역', '서비스', 'Subup', '및', '...","['어떤', '업무', '를', '하게', '되나요', '?', '알트', '에디터..."
3,유섭,"['백엔드', '개발자', 'Backend', 'Software', 'Enginee...",웹 개발자,"['3년', '이상', '또는', '그', '에', '준', '하는', '백엔드',...","['이런', '분', '은', '더욱', '환영', '합니다', '.', '팀', ...",3,https://www.wanted.co.kr/wd/167133,"['자유로운', '휴가', '무제한', '무급', '휴가', '동료', '와', '...","['유섭', '은', '현재', '번역', '서비스', 'Subup', '및', '...","['어떤', '업무', '를', '하게', '되나요', '?', '알트', '에디터..."
4,인투씨엔에스,"['웹', '개발', '인재', 'PHP']",웹 개발자,"['PHP', '중', '급', '이상', 'MYSQL', '중', '급', '이상...","['웹', '서비스', '라이브', '런칭', '경험', '자', 'RESTful'...",0,https://www.wanted.co.kr/wd/166922,"['매월', '1회', '패밀리', '데이', '오전', '근무', '세번', '째...","['다음', '과', '같이', '유능하고', '역량', '있는', '인재', '를...","['웹', '기반', '검사', '시스템', '개발', '및', '유지', '보수'..."
5,꾸까(kukka),"['E', 'commerce', '백엔드', '개발자']",웹 개발자,"['1년', '이상', '의', '백엔드', '개발', '경력', '또는', '그'...","['온라인', '커머스', '서비스', '업계', '경험', '이', '있으신', ...",1,https://www.wanted.co.kr/wd/166844,"['서울', '7', '호선', '내방역', '1', '번', '출구', '도보',...","['플라워', '대표', '브랜드', '꾸까', 'kukka', '에서', '함께'...","['꾸까', 'e', '커머스', '전용', '프레임워크', '개발', '및', '..."
6,베이스라인파트너스,"['Cloud', 'AWS', 'GCP', '기반', '운영', '및', '개발',...",웹 개발자,"['AWS', 'linux', 'gitLab', 'DB', 'mysql', 'EKS...","['AWS', '운영', '경험', '개발', 'python', 'java', '등...",0,https://www.wanted.co.kr/wd/166616,"['4', '대', '보험', '연차', '제도', '경', '조휴', '가', '...","['""', '같이의', '가치', '를', '아는', '기업', '""', '베이스'...","['DB', '단', '개발', '및', '데이터', '관리', '필수', '제출'..."
7,법틀,"['SW', '웹', 'Back', 'End', 'Front', 'PL', '개발자...",웹 개발자,"['7년', '이상', '의', '소프트웨어', '개발자', '경력', 'Junio...","['Refactoring', '경험', 'ORM', 'DB', '모델링', '설계'...",7,https://www.wanted.co.kr/wd/166030,"['1', '편한', '근무', '를', '지원', '합니다', '.', '일', ...","['B', '2', 'B', 'Legal', 'Cloud', 'SAAS', '의',...","['Python', 'Django', 'Java', 'Spring', 'Boot',..."
8,심심이,"['산업', '기능', '요원', '보충역', '신입', '만', '개발자']",웹 개발자,"['AWS', '등', '클라우드', '컴퓨팅', '경험', '자', 'chatGP...","['Android', 'iOS', 'React', 'Native', '로', '설계...",0,https://www.wanted.co.kr/wd/165816,"['야근', '은', '절대', '지', '양', '근무시간', '내', '에', ...","['보충역', '신규', '편입', '자만', '지원', '가능합니다', '.', ...","['다양한', '정부', '과제', '프로젝트', '챗봇', '데이터', '구축',..."
9,이지스엔터프라이즈,"['프론트엔드', '개발자', 'VUEJS', '_', '시니어']",웹 개발자,"['기본', '기술', '자격', '요건', 'Vue', '.', 'js', '개발...","['보유', '기술', '우대', '사항', 'Vue', '.', 'js', '+'...",6,https://www.wanted.co.kr/wd/164753,"['건강한', '사내', '문화', '탄력', '근', '무제', '자유로운', '...","['압도', '적', '인', '안정', '성과', '시장', '점유', '율', ...","['이지스', '앤터', '프라', '이즈', '에서', '제공', '하는', '공..."


## 최종 result 데이터 저장


In [21]:
result.to_csv('./Data_result/pre_result.csv', index=False)